In [2]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd


def get_soup(url):

    header = {
        "User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.108 Safari/537.36Name"
    }

    content = requests.get(url , headers = header).text

    soup = BeautifulSoup(content, "html.parser")
    
    return soup

# import re
# def striphtml(data):
#     clean = re.compile('<.*?>')
#     clean = re.sub(clean, ' ', data)
#     clean = clean.replace(u'\xa0', u' ')
#     return clean

def splithtml(data):
   
    newclean=[]
    clean =  re.split(r'<.*?>', data)
    clean = [data.replace(u'\xa0', u' ') for data in clean]
    clean2 = [x for x in clean if x.strip()]
    return clean2



In [25]:
url_prefix = "https://jobs.msd.com"
url_list=[]

for n in np.arange(0,1000,25):
    searchlisting_url = url_prefix+"/search/?q=&startrow="+str(n)
    listingsoup = get_soup(searchlisting_url)
    jobpostings= listingsoup.find_all("a", {"class": "jobTitle-link"},href=True)
    
    for posting in jobpostings:
        url = url_prefix+posting['href']
        url_list.append(url)


In [28]:
with open('msdjoblist.txt', 'w') as f:
    for item in url_list:
        f.write("%s\n" % item)
        

In [37]:
with open('msdjoblist.txt', 'r') as f:
   
    x = f.readlines()
    
url_list = [url.strip() for url in x]

In [50]:
jobinfo=[]

In [62]:

for url in url_list[400:500]:
    soup = get_soup(url)
    postInfo = {}
    postInfo['url']=url
    postInfo["title"] = soup.find("title").text
    postInfo["datePosted"] = soup.find('span',{'itemprop':"datePosted"}).text.strip()
    postInfo["similarJobs"] = [entry['title'] for entry in soup.find('div',{'id':'similar-jobs-links'}).find_all('a')]
    postInfo["location"] = soup.find('span',{'itemprop':"jobLocation"}).text.strip()

    
    description = splithtml(str(soup.find('span',{'itemprop':"description"})))

    keys = ['Requisition ID', 'Job',]

    stopwords= ['Other Locations', 'Employee Status', 'Travel','Number of Openings','Shift (if applicable)',
    'Hazardous Materials','Company Trade Name', 'Regular']


    textdescription = []

    i=0
    while i < len(description):
        if description[i].replace(":","").strip() in keys:
            postInfo[description[i].replace(":","").strip()] = description[i+1].strip()
            i+=2
        else:
            textdescription.append(description[i].strip())
            i+=1
    postInfo['description'] = textdescription
    
    

    jobinfo.append(postInfo)

In [59]:
#scrape url 500 onwards
len(jobinfo)

400

In [60]:
df = pd.DataFrame(jobinfo)
df.to_csv('data.csv')

In [61]:
df

,Job,Requisition ID,datePosted,description,location,similarJobs,title,url
0,Engineering Generic,ENG004029,13.03.2019,[Unsere Manufacturing Division ist ein Team vo...,"Krems, 3, AT","[Operations and Production Jobs, MSD Manufactu...",Automatisierungsingenieur (m/w),https://jobs.msd.com/MSD/job/Krems-Automatisie...
1,Project Management Generic,PRO017569,"Mar 13, 2019",[Mit seinen verschreibungspflichtigen Arzneimi...,"Vienna, 9, AT","[Scientific Jobs, Project Management & Directo...",Facility Manager (m/w) Job,https://jobs.msd.com/MSD/job/Vienna-Facility-M...
2,Quality Operations,QUA007972,"Mar 13, 2019",[Do you enjoy working in an international envi...,"Boxmeer, NB, NL","[Scientific Jobs, Project Management & Directo...",Quality Officer Job,https://jobs.msd.com/MSD/job/Boxmeer-Systems-&...
3,Financial Analysis,FIN002035,13 mars 2019,[MSD un des leaders mondiaux de l’industrie ph...,"Courbevoie, J, FR","[Operations and Production Jobs, Marketing Job...",Analyste Prévisions de Ventes – H/F,https://jobs.msd.com/MSD/job/Courbevoie-Analys...
4,Product Management/Marketing,PRO019126,"Mar 13, 2019",[Position Overview - Basic Functions &amp; Res...,"Sydney, NS, AU","[Sales Jobs, Regulatory Affairs Jobs, Project ...",Senior Brand Manager (Diabetes) Job,https://jobs.msd.com/MSD/job/Sydney-Senior-Bra...
5,Product Management/Marketing,PRO019168,"Mar 13, 2019",[MSD is a global health care leader with a div...,"Warsaw, PL","[Regulatory Affairs Jobs, Project Management &...",Diversified Brands Lead Job,https://jobs.msd.com/MSD/job/Warsaw-Diversifie...
6,Engineering Generic,ENG004042,"Mar 13, 2019","[MSD BIOTECH DUBLIN, Are you ready to Invent, ...","Dublin, IE","[Sales Jobs, Regulatory Affairs Jobs, Pennsylv...",Principal Scientist – Upstream Process Lead (D...,https://jobs.msd.com/MSD/job/Dublin-Principal-...
7,External Communications,EXT000415,13.03.2019,[Mit seinen verschreibungspflichtigen Arzneimi...,"Haar, BY, DE","[Sales Jobs, Project Management & Director Job...",Project Manager (m/w/d) Digital Customer Access,https://jobs.msd.com/MSD/job/Haar-Project-Mana...
8,Information Technology Architecture,INF004840,13.3.2019,[MSD is a global health care leader with a div...,"Prague, CZ","[Veterinary Medicine Jobs, Project Management ...","Technical Lead – Integration, Big Data",https://jobs.msd.com/MSD/job/Prague-Technical-...
9,Epidemiology,EPI000283,"Mar 13, 2019",[Under the general direction of the Head-Quart...,"Beijing, 11, CN","[ Pharmaceutical Jobs, Pennsylvania Operations...","Director, China Pharmacoepidemiology Job",https://jobs.msd.com/MSD/job/Beijing-Director%...


In [24]:
postInfo

{'Job': 'Product Management/Marketing',
 'Requisition ID': 'PRO019126',
 'datePosted': 'Mar 13, 2019',
 'description': ['Position Overview - Basic Functions &amp; Responsibility',
  'Description',
  '&amp; Role Purpose',
  'Organizational Environmen',
  't',
  'The Senior Brand Manager is responsible for overseeing and implementing the effective short and long term initiatives supporting Brand strategies.',
  'There is a strong focus on longer term strategic planning, identifying and cementing synergies across brands and external partnership opportunities.',
  'The Senior Brand Manager role sits within a Product Life Cycle Planning team in the Sales &amp; Marketing organization and reports to either the Business Unit Manager or Marketing Manager, which in turn reports to the Sales &amp; Marketing Director.',
  'The Senior Brand Manager role may/may not have people management responsibility and will require additional activities, knowledge, skills and experience.',
  'In this case the r

In [7]:
test2=splithtml(str(test))

In [10]:
textdescription

['Position Overview - Basic Functions &amp; Responsibility',
 'Description',
 '&amp; Role Purpose',
 'Organizational Environmen',
 't',
 'The Senior Brand Manager is responsible for overseeing and implementing the effective short and long term initiatives supporting Brand strategies.',
 'There is a strong focus on longer term strategic planning, identifying and cementing synergies across brands and external partnership opportunities.',
 'The Senior Brand Manager role sits within a Product Life Cycle Planning team in the Sales &amp; Marketing organization and reports to either the Business Unit Manager or Marketing Manager, which in turn reports to the Sales &amp; Marketing Director.',
 'The Senior Brand Manager role may/may not have people management responsibility and will require additional activities, knowledge, skills and experience.',
 'In this case the role also supervises the activities of marketing staff as required to develop and extend individual skills and ensure marketing o

In [67]:
test = soup.find('span',{'itemprop':"description"})
tags = test.find_all('b')
# for entry in test.find_all('b'):
# #     print(test.find(entry))
# test1= test.find('Qualifications')  
# test1
test

<span class="jobdescription" itemprop="description"><b>Requisition ID: </b>REL000408<br/><br/><font face="Arial" size="3"><p style="MARGIN-BOTTOM: 0px; MARGIN-TOP: 0px"><i>Do you enjoy working in an international environment with lots of room for personal development? Did you always want to work in a company where your work can have a real impact on people’s life? Would you like to change and drive things?</i></p><br/><p style="MARGIN-BOTTOM: 0px; MARGIN-TOP: 0px"><br/></p><p style="MARGIN-BOTTOM: 0px; MARGIN-TOP: 0px">We have an exciting opportunity for a <b>Quality Operations Manager</b></p><br><p style="MARGIN-BOTTOM: 0px; MARGIN-TOP: 0px"></p><br/><p style="MARGIN-BOTTOM: 0px; MARGIN-TOP: 0px"></p><br/><p style="MARGIN-BOTTOM: 0px; MARGIN-TOP: 0px"><br/></p><p style="MARGIN-BOTTOM: 0px; MARGIN-TOP: 0px">As an Quality Operations Manager at MSD you are responsible for the operational management and realization of planning at the Quality Release Blister/Specialties department.  In thi

In [47]:
# sections=[]
# sections = [sectionName.text.replace(":","").strip() for sectionName in desc.find_all('p')]
# #     sectionInfo = desc.find_all('ul')
# #     for section in sectionInfo:
# #         print(section.find_parent('b'))
# #     for var,info in zip(sections,sectionInfo):
# #         postInfo[var] = [listitem.text.strip() for listitem in info.find_all('li')]
    
# #     jobinfo.append(postInfo)
# desc.find_all('p')

In [92]:

# for section in desc.find_all('ul'):
    
#     sectionName = section.previous_sibling.find('b').text
#     print(sectionName)
#     postInfo[sectionName] = [listitem.text.strip() for listitem in section.find_all('li')]
#     print('\n')

AttributeError: 'NoneType' object has no attribute 'text'

In [156]:
sections=desc.find_all('p')

for section in sections:
    if section.find('ul'):
        #sectionName = section.find('b').text
        print(section)
        print("NEXT")
        #postInfo[sectionName] = [listitem.text.strip() for listitem in section.find_all('li')]
        


<p><p><b>Responsibilities</b></p><p><p><b>REGION MARKETING STRATEGY &amp; STRATEGIC PLANNING</b></p><p>Define, refine and ensure the implementation of vaccine portfolio strategies and objectives in line with regional ambition and needs. The goal being to continually improve market position with consistency and coherency across the product ranges for both Private and Public markets. Ensure proper integration of portfolio strategies through:</p><ul><li>Country Strategic Planning exercise</li><li>Effective Brand Plans within each country</li><li>Budget &amp; 3 year Financial Forecast exercise</li></ul><p>Drive, implement and monitor key Global and Regional strategies/initiatives through strategic input and close collaboration with the countries and across regional functions</p><p>Build and defend the necessary resources and capabilities at both regional and country level to achieve sales ambition through:</p><ul><li>Regional Execution Plan</li><li>Delivering expertise and training to guid

In [139]:
for section in sections:
    if section.find('ul'):
        print(section.find('b').text)

Purpose of Role  
Priorities
Person Profile
Person Profile
Essential Skills and Experience
Essential Qualifications
Desirable Skills and Experience


In [147]:
import pandas as pd
df = pd.DataFrame(jobinfo)

In [148]:
df

,Act as PMO MCE :,BUSINESS PROCESSES:,COMMERCIAL EXCELLENCE,Challenges:,Desirable Qualifications,Desirable Skills and Experience,Develop Marketing Excellence,Digital Innovation,Education Qualification,Education:,...,Requirements & Qualifications,Responsibilities,SALES & MARKETING EXCELLENCE,Skills & Core Competencies,Strategy Execution to Maximize Revenues,contractType,datePosted,jobID,jobLocation,role
0,NaN,NaN,NaN,NaN,NaN,[Sales experience in dealing with niche biopha...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Full time,02/01/2019,R2492720,Singapore,Clinical Science Associate at Sanofi Global (E...
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,[Manage the annual medical affairs budget for ...,NaN,NaN,NaN,Full time,01/08/2019,R2486999,Singapore,"Asia Medical Operations Lead, GEM at Sanofi Gl..."
2,[Expedite adaptation of MCE/Digital applied kn...,NaN,NaN,NaN,NaN,NaN,NaN,[Digital Innovation Understanding and applicat...,"[Bachelor degree in a related field, eg Market...",NaN,...,NaN,NaN,NaN,[Multi-Channel Engagement and Digital competen...,[Strategy Execution to Maximize Revenues Actio...,Full time,01/08/2019,R2484967,Singapore,"PMO, Asia Multi-Channel Engagement at Sanofi G..."
3,NaN,NaN,NaN,NaN,"[Medical Degree; and/or PhD, Post graduate deg...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Full time,10/11/2018,R2478311,Singapore,"Regional Medical Expert - Influenza, Asia and ..."
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Full time,08/10/2018,R2471961,Singapore,Chemical Technician at Sanofi Global (English)
5,NaN,NaN,NaN,[MSc or PhD in Epidemiology/Mathematics /Econo...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,[Develop the region’s evidence generation plan...,NaN,NaN,NaN,Full time,06/08/2018,R2462075,Singapore,"Head, Epidemiology and Health Economics, Asia ..."
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"[Bachelor's Degree in Marketing, Communication...",[Lead the implementation of TA Marketing initi...,NaN,NaN,NaN,Full time,05/08/2018,R2455428,Singapore,Talent Acquisition Marketing Manager - Asia & ...
7,NaN,NaN,[Bachelor’s degree qualification with Post Gra...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,"[Country Strategic Planning exercise, Effectiv...",[Bachelor’s degree qualification with Post Gra...,NaN,NaN,Full time,02/06/2018,R2442562,Singapore,"Regional Head of Marketing, Vaccines - ASIA & ..."
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Full time,11/08/2017,R22300,Singapore,Regional Digital & Multi-Channel Marketing Man...
9,NaN,[Provide LRP/CSP/ Budget support to the countr...,NaN,NaN,NaN,NaN,[Ensure right product knowledge by all means (...,NaN,NaN,"[University degree, preferably from a scientif...",...,NaN,NaN,NaN,NaN,NaN,Full time,02/27/2019,R2494096,Singapore,REGIONAL MARKETING MANAGER – MENINGITIS & NEW ...


In [53]:

    
# for section in sections:
#     section.find_all('li')

In [56]:
len(jobinfo['Requirements'])

10

In [7]:
len(job_details_urls)

10

In [9]:
for details_url in job_details_urls[0:20]:
    #print details_url
    soup = getsoup(details_url[-1])
    
    #get the title
    title_box = soup.find("div", {"class": "title-page"})
    if title_box:
        job_title = title_box.find("h3")
        #print job_title.text
        
    
        #get the company name
        company_name_h4 = title_box.find("h4")
        company_details = company_name_h4.text.split("Posted On :")
        company_name = company_details[0].strip()
        posted_on = company_details[1].strip()
        
        #print (job_title.text, company_name,posted_on)
        
        
        #COMPANY DETAILS
        company_info_ul = soup.find("ul", {"class": "who"})
        
        #if company_info_ul:
            #print company_info_ul.findChild().text
        
        address = ""
        for a in company_info_ul.find_all("a"):
            address = address + a.text.strip() + ","
        address = address[:-1]
        #print address
        
        print (job_title.text, company_name,posted_on,address)
    
    

(u'Software Engineer ', u'Streetsine Singapore Pte Ltd', u'12 Jan 2018', u'http://www.srx.com.sg/')
(u'Software Engineers/ Senior Software Engineers (Digital Initiatives) ', u'SPH Digital', u'12 Jan 2018', u'1000 TOA PAYOH NORTH NEWS CENTRE Singapore 318994,http://www.sphdigital.com')
(u'Software Engineers - Swiss Medical Devices industrial leader', u'MICHAEL PAGE INTERNATIONAL (SINGAPORE)', u'12 Jan 2018', u'1 RAFFLES PLACE, #09-61 ONE RAFFLES PLACE Singapore 048616,http://www.michaelpage.com.sg,EA Licence No: 98C5473')
(u'Software Engineer [ C++ / Computer Engineering / Training Provided!]', u'The Supreme HR Advisory Pte Ltd', u'12 Jan 2018', u'http://www.thesupremehr.com,EA Licence No: 14C7279,Personnel Reg. No: R1657954')
(u'Software Engineer - C#.Net ', u'MICHAEL PAGE INTERNATIONAL (SINGAPORE)', u'11 Jan 2018', u'1 RAFFLES PLACE, #09-61 ONE RAFFLES PLACE Singapore 048616,http://www.michaelpage.com.sg,EA Licence No: 98C5473')
(u'Software/System Engineer ( (In-house / West / S$3K++)

In [10]:
print company_info_ul.findChild

<bound method Tag.find of <ul class="list-unstyled who">\n<li style="text-transform: uppercase;"><strong>The Supreme HR Advisory Pte Ltd</strong>\n</li>\n<li><a href="http://www.thesupremehr.com" target="_blank"><i class="fa fa-globe"></i> http://www.thesupremehr.com</a></li>\n<li><a href="#"><i class="fa fa-info"></i> EA Licence No: 14C7279</a></li>\n<li><a href="#"><i class="fa fa-user"></i> Personnel Reg. No: R1764404</a></li>\n</ul>>
